In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from ssl import create_default_context
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime
from datetime import timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [ ]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy, es usada para concatenar al nombre del indice principal previa inserción
* ahora, es la fecha en un formato requerido para ES

In [ ]:
now = datetime.now()
format_ES = "%Y.%m.%d"
fecha_hoy = str(now.strftime(format_ES))
ahora_format = "%Y-%m-%d"'T'"%H:%M:%S"
ahora = str(now.strftime(ahora_format))

### Definiendo indice principal con fecha de hoy

In [ ]:
indice = parametros.gestion_tablero16_index 
indice_control = parametros.tableros_mintic_control

In [ ]:
indice

### Función para generar JSON compatible con ES

In [ ]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [ ]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
           "_source": ["tablero16.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero16"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['tablero16.fechaControl']
except Exception as e:
    print(e)
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-14T00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)



### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede.)

In [ ]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
                           ,'nombreSede','latitud','longitud','id_Beneficiario','COD_ISO','codDanesede',
                           'cod_servicio','codDaneMuni','nombre_centro_pob','codCentroPoblado','codDaneInstitucionEdu',
                           'tipoSitio','detalleSitio','energia','region','matricula','DDA','grupoDesc','estadoInstalacion',
                           'nombreInstitucionEd']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except Exception as e:
    print(e)
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()

In [ ]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'
    
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['gestion.tablero16.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['gestion.tablero16.location']=datos_semilla['gestion.tablero16.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)


### Leyendo indice servicemanager-interacciones

Se lee la información de los De Service Manager. En la lectura se traen todas las interacciones reportadas por servicemanager, con la información de la misma.

* id_beneficiario, es la llave para cruzar con cada centro de conexión.
* variable1 y variable2, son la tipo y subtipo de la interaccion.
* current_phase, estado del ticket
* clr_txt_assignment, es el sujeto asignado al ticket
* owner_name, usuario del ticket
* open_time y close_time, son las fecha de apertura y cierre del ticket
* resolution, respuesta al ticket
* incident_id, numero del ticket
* source, fuente del ticket(email o telefono)
* severity, gravedad del ticket
* clr_txt_codcategorymtc categoria segun mintic

In [ ]:
def traeSMInteracciones(fecha_max,fecha_tope):
    total_docs = 10000
    response = es.search(
        index= parametros.sm_interaciones,
        body={
            "_source": ['variable1'
                  , 'opened_by'
                  , 'subcategory'
                  , '@timestamp'
                  , 'open_time'
                  , 'clr_txt_idbneficmtc'
                  , 'category'
                  , 'close_time'
                  , 'source'
                  , 'key_char'
                  , 'total'
                  , 'close_date'
                  , 'resolution_code'
                  , 'resolved_time'
                  , 'callback_type'
                  , 'initial_impact'
                  , 'name'
                  , 'variable2'
                  , '@version'
                  , 'contact_name'
                  , 'variable3'
                        ,'total_claro'
                        ]
            ,"query": {
              "range": {
                "open_time": {
                    "gte": fecha_max,
                    "lt": fecha_tope
                }
              }
          }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_SM_interactions = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    
    if datos_SM_interactions is None or datos_SM_interactions.empty:
        return pd.DataFrame()
    
    datos_SM_interactions = datos_SM_interactions.rename(columns={'variable1':'gestion.tablero16.categoria'
                                                                  , 'opened_by':'gestion.tablero16.opened_by'
                                                                  , 'subcategory':'gestion.tablero16.subcategoria'
                                                                  , '@timestamp':'gestion.tablero16.@timestamp'
                                                                  , 'open_time':'gestion.tablero16.fechaApertura'
                                                                  , 'clr_txt_idbneficmtc':'gestion.tablero16.id_Beneficiario'
                                                                  , 'category':'gestion.tablero16.category'
                                                                  , 'close_time':'gestion.tablero16.fechaCierre'
                                                                  , 'source':'gestion.tablero16.canal'
                                                                  , 'key_char':'gestion.tablero16.numeroTicket'
                                                                  , 'total':'gestion.tablero16.total'
                                                                  , 'close_date':'gestion.tablero16.close_date'
                                                                  , 'resolution_code':'gestion.tablero16.coderesolucion'
                                                                  , 'resolved_time':'gestion.tablero16.time-resolution'
                                                                  , 'callback_type':'gestion.tablero16.callback_type'
                                                                  , 'initial_impact':'gestion.tablero16.initial_impact'
                                                                  , 'name':'gestion.tablero16.name'
                                                                  , 'variable2':'gestion.tablero16.variable2'
                                                                  , '@version':'gestion.tablero16.version'
                                                                  , 'contact_name':'gestion.tablero16.contact_name'                                                                  
                                                                  , 'variable3':'gestion.tablero16.variable3'})         
    datos_SM_interactions = datos_SM_interactions.dropna(subset=['gestion.tablero16.id_Beneficiario'])
    datos_SM_interactions = datos_SM_interactions.drop(datos_SM_interactions[datos_SM_interactions['gestion.tablero16.id_Beneficiario']=="null"].index)
    datos_SM_interactions['gestion.tablero16.id_Beneficiario'] = datos_SM_interactions['gestion.tablero16.id_Beneficiario'].astype(int)
    datos_SM_interactions['gestion.tablero16.canal'] = datos_SM_interactions['gestion.tablero16.canal'].replace(['5','6'],['Correo Electrónico','Teléfono'])
    return datos_SM_interactions

Realizando bucle hasta conseguir datos de servicemanager-interacciones o hasta la fecha actual para realizar la carga de datos 

In [ ]:
fecha_max_mintic = fecha_ejecucion



fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
datos_SM_interactions = traeSMInteracciones(fecha_max_mintic,fecha_tope_mintic)


if datos_SM_interactions is None or datos_SM_interactions.empty:
    while (datos_SM_interactions is None or datos_SM_interactions.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d"'T'"%H:%M:%S")) < str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        datos_SM_interactions = traeSMInteracciones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass



In [ ]:
#fecha_max_mintic
#print (fecha_max_mintic, "   hastaaaa  ", fecha_tope_mintic)

Haciendo merge entre semilla e interacciones

Realizando inserción

In [ ]:
use_these_keys = [ 'gestion.tablero16.nombreSede'
                    , 'gestion.tablero16.DDA'
                    , 'gestion.tablero16.estadoInstalacion' 
                    , 'gestion.tablero16.COD_ISO'
                    , 'gestion.tablero16.energia'
                    , 'gestion.tablero16.departamento'
                    , 'gestion.tablero16.codCentroPoblado'
                    , 'gestion.tablero16.nombreInstitucionEd'
                    , 'gestion.tablero16.codDanesede'
                    , 'gestion.tablero16.tipoSitio'
                    , 'gestion.tablero16.codDaneMuni'
                    , 'gestion.tablero16.nombre_centro_pob'
                    , 'gestion.tablero16.site_id'
                    , 'gestion.tablero16.matricula'
                    , 'gestion.tablero16.energiadesc'
                    , 'gestion.tablero16.grupoDesc'
                    , 'gestion.tablero16.cod_servicio'
                    , 'gestion.tablero16.region'
                    , 'gestion.tablero16.detalleSitio'
                    , 'gestion.tablero16.municipio'
                    , 'gestion.tablero16.id_Beneficiario'
                    , 'gestion.tablero16.location'
                    , 'gestion.tablero16.codDaneInstitucionEdu'
                    , 'gestion.tablero16.categoria'
                    #, 'gestion.tablero16.opened_by'
                    , 'gestion.tablero16.subcategoria'
                    , 'gestion.tablero16.@timestamp'
                    , 'gestion.tablero16.fechaApertura'
                    , 'gestion.tablero16.id_Beneficiario'
                    , 'gestion.tablero16.category'
                    , 'gestion.tablero16.fechaCierre'
                    , 'gestion.tablero16.canal'
                    , 'gestion.tablero16.numeroTicket'
                    #, 'gestion.tablero16.total'
                    , 'gestion.tablero16.close_date'
                    # , 'gestion.tablero16.coderesolucion'
                    , 'gestion.tablero16.time-resolution'                                                                                       
                    , 'gestion.tablero16.callback_type'
                    , 'gestion.tablero16.initial_impact'
                    , 'gestion.tablero16.name'
                    , 'gestion.tablero16.variable2'
                    , 'gestion.tablero16.version'
                    #, 'gestion.tablero16.contact_name'                                                                  
                    , 'gestion.tablero16.variable3'
                    , 'gestion.tablero16.anyo'
                    , 'gestion.tablero16.mes'
                    , 'gestion.tablero16.dia'
                  ]
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{str(document['gestion.tablero16.site_id']) + '-' + str(document['gestion.tablero16.numeroTicket']) + '-' + str(document['gestion.tablero16.fechaApertura'])+ str(random.randrange(1000))}",
                "_source": filterKeys(document),
            }

In [ ]:
def doc_ordenamiento(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": 1,
                "_source": filterKeys(document),
            }

In [ ]:
datos_semilla.columns=['nombreSede', 'DDA', 'estadoInstalacion', 'COD_ISO', 'energia',
'nombre_departamento', 'codCentroPoblado', 'nombreInstitucionEd',
'codDanesede', 'tipoSitio', 'codDaneMuni', 'nombre_centro_pob',
'site_id', 'matricula', 'energiadesc', 'grupoDesc', 'cod_servicio',
'region', 'detalleSitio', 'nombre_municipio', 'gestion.tablero16.id_Beneficiario',
'codDaneInstitucionEdu', 'gestion.tablero16.location']

In [31]:
try:
    concat = pd.merge(datos_SM_interactions,datos_semilla, on=['gestion.tablero16.id_Beneficiario'],how='inner')
    concat['gestion.tablero16.fechaApertura'] = concat['gestion.tablero16.fechaApertura'].str.replace("T"," ")
    concat['gestion.tablero16.fechaApertura'] = concat['gestion.tablero16.fechaApertura'].str.slice(stop=19)
    concat["gestion.tablero16.anyo"] = concat["gestion.tablero16.fechaApertura"].str[0:4]
    concat["gestion.tablero16.mes"] = concat["gestion.tablero16.fechaApertura"].str[5:7]
    concat["gestion.tablero16.dia"] = concat["gestion.tablero16.fechaApertura"].str[8:10]
    concat['@timestamp'] = now.isoformat()
    
    
    concat = concat.fillna(0)
             
    salida = helpers.bulk(es, doc_generator(concat))
    print("Fecha: ", now,"- Interacciones insertadas en indice principal:",salida[0])
except Exception as e:
    print(e, "\nNinguna Interacción insertada en indice principal:")

'site_id' 
Ninguna Interacción insertada en indice principal:


Actualizando fecha de control de ejecución

In [32]:
concat

,gestion.tablero16.subcategoria,gestion.tablero16.gravedad,gestion.tablero16.tipo3,variable2,gestion.tablero16.usuarioTicket,gestion.tablero16.estadoTicket,gestion.tablero16.fechaApertura,gestion.tablero16.canal,gestion.tablero16.fechaCierre,gestion.tablero16.resolucion,...,gestion.tablero16.cod_servicio,gestion.tablero16.region,gestion.tablero16.detalleSitio,gestion.tablero16.municipio,gestion.tablero16.codDaneInstitucionEdu,gestion.tablero16.location,gestion.tablero16.anyo,gestion.tablero16.mes,gestion.tablero16.dia,@timestamp
0,Navegacion Internet,1,Navegacion Internet / Sin acceso a internet,Sin acceso a internet,Emiliano Rafeal Serrano Perez,Work In Progress,2021-07-01 00:03:08,Teléfono,null,null,...,ZGYO122,A,8,VALLEDUPAR,220001002791,"10.085547,-73.68832614",2021,07,01,2021-08-12T18:49:30.868295
1,Navegacion Internet,1,Navegacion Internet / Sin acceso a internet,Sin acceso a internet,Emiliano Rafeal Serrano Perez,Work In Progress,2021-07-01 00:03:08,Teléfono,null,null,...,ZGYO122,A,8,VALLEDUPAR,220001002791,"10.085547,-73.68832614",2021,07,01,2021-08-12T18:49:30.868295
2,Navegacion Internet,1,Navegacion Internet / Sin acceso a internet,Sin acceso a internet,Emiliano Rafeal Serrano Perez,Work In Progress,2021-07-01 00:03:08,Teléfono,null,null,...,ZGYO122,A,8,VALLEDUPAR,220001002791,"10.085547,-73.68832614",2021,07,01,2021-08-12T18:49:30.868295
3,Navegacion Internet,1,Navegacion Internet / Sin acceso a internet,Sin acceso a internet,Emiliano Rafeal Serrano Perez,Work In Progress,2021-07-01 00:03:08,Teléfono,null,null,...,ZGYO122,A,8,VALLEDUPAR,220001002791,"10.085547,-73.68832614",2021,07,01,2021-08-12T18:49:30.868295
4,Navegacion Internet,1,Navegacion Internet / Sin acceso a internet,Sin acceso a internet,Emiliano Rafeal Serrano Perez,Work In Progress,2021-07-01 00:03:08,Teléfono,null,null,...,ZGYO122,A,8,VALLEDUPAR,220001002791,"10.085547,-73.68832614",2021,07,01,2021-08-12T18:49:30.868295
5,Navegacion Internet,1,Navegacion Internet / Sin acceso a internet,Sin acceso a internet,Emiliano Rafeal Serrano Perez,Work In Progress,2021-07-01 00:03:08,Teléfono,null,null,...,ZGYO122,A,8,VALLEDUPAR,220001002791,"10.085547,-73.68832614",2021,07,01,2021-08-12T18:49:30.868295
6,Navegacion Internet,1,Navegacion Internet / Sin acceso a internet,Sin acceso a internet,Emiliano Rafeal Serrano Perez,Work In Progress,2021-07-01 00:03:08,Teléfono,null,null,...,ZGYO122,A,8,VALLEDUPAR,220001002791,"10.085547,-73.68832614",2021,07,01,2021-08-12T18:49:30.868295
7,Navegacion Internet,1,Navegacion Internet / Sin acceso a internet,Sin acceso a internet,null,null,2021-07-01 00:03:08,null,2021-07-21T15:43:28.000Z,null,...,ZGYO122,A,8,VALLEDUPAR,220001002791,"10.085547,-73.68832614",2021,07,01,2021-08-12T18:49:30.868295
8,Navegacion Internet,1,Navegacion Internet / Sin acceso a internet,Sin acceso a internet,null,null,2021-07-01 00:03:08,null,2021-07-21T15:43:28.000Z,null,...,ZGYO122,A,8,VALLEDUPAR,220001002791,"10.085547,-73.68832614",2021,07,01,2021-08-12T18:49:30.868295
9,Navegacion Internet,1,Navegacion Internet / Sin acceso a internet,Sin acceso a internet,null,null,2021-07-01 00:03:08,null,2021-07-21T15:43:28.000Z,null,...,ZGYO122,A,8,VALLEDUPAR,220001002791,"10.085547,-73.68832614",2021,07,01,2021-08-12T18:49:30.868295


In [33]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=50)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    
#fecha_ejecucion = '2021-04-29T00:00:00'

if fecha_ejecucion > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-tablero16',
        body = { 'jerarquia-tablero16': 'jerarquia-tablero16','tablero16.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)


actualizada fecha control de ejecucion: 2021-07-01T00:10:00
